In [5]:
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
import hdbscan

In [2]:
df = pd.read_csv('ml_data/train.csv')
df.head(5)

,URI,Title,Year,Plot,Director,Genre1,Genre2,Genre3,Country,RatingAvg,RatingCount,MyRating
0,https://boxd.it/3d1m,Transformers: Age of Extinction,2014,As humanity picks up the pieces after the batt...,Michael Bay,Science Fiction,Adventure,Action,USA,2.12,235666.0,1.0
1,https://boxd.it/2NF6,Madagascar 3: Europe's Most Wanted,2012,"Animal pals Alex, Marty, Melman, and Gloria ar...",Eric Darnell,Comedy,Family,Adventure,USA,3.03,296110.0,3.0
2,https://boxd.it/fx08,The First Omen,2024,When a young American woman is sent to Rome to...,Arkasha Stevenson,Horror,NaN,NaN,USA,3.38,297652.0,3.5
3,https://boxd.it/2abQ,Terminator Salvation,2009,"All grown up in post-apocalyptic 2018, John Co...",McG,Thriller,Action,Science Fiction,USA,2.58,146188.0,2.5
4,https://boxd.it/mpWM,I Hate Summer,2020,Three families end up in the same rented house...,Massimo Venier,Comedy,NaN,NaN,Italy,3.25,7717.0,3.0


In [65]:
# 1. Embedding model
sbert_model = SentenceTransformer("all-MiniLM-L6-v2")

# 2. CountVectorizer per il c-TF-IDF
vectorizer = CountVectorizer(
    stop_words="english",
    max_df=0.85,
    min_df=5
)

# 2. UMAP più globale e compatto
umap_model = UMAP(
    n_neighbors=75,     # scala globale→ outlier più vicini ai cluster
    n_components=20,     # dimensione ridotta “leggera”
    min_dist=0.0,       # punti molto compatti nel low-dim
    metric="cosine",
    random_state=42
)

# 3. HDBSCAN permissivo sui core points, “leaf” per meno outlier
hdbscan_model = hdbscan.HDBSCAN(
    min_cluster_size=20,       # cluster ≥ 20
    min_samples=1,             # densità minima più bassa possibile
    cluster_selection_method="leaf",
    prediction_data=True
)


# 4. Costruzione del topic model
topic_model = BERTopic(
    embedding_model=sbert_model,
    vectorizer_model=vectorizer,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    nr_topics="auto",
    calculate_probabilities=True,
    verbose=True
)

In [66]:
# 4. Fit + transform
docs = df["Plot"].tolist()
topics, probs = topic_model.fit_transform(docs)

2025-05-25 20:04:18,007 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/44 [00:00<?, ?it/s]

2025-05-25 20:04:27,464 - BERTopic - Embedding - Completed ✓
2025-05-25 20:04:27,465 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-05-25 20:04:33,915 - BERTopic - Dimensionality - Completed ✓
2025-05-25 20:04:33,915 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-05-25 20:04:34,049 - BERTopic - Cluster - Completed ✓
2025-05-25 20:04:34,051 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2025-05-25 20:04:34,110 - BERTopic - Representation - Completed ✓
2025-05-25 20:04:34,111 - BERTopic - Topic reduction - Reducing number of topics
2025-05-25 20:04:34,115 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-05-25 20:04:34,166 - BERTopic - Representation - Completed ✓
2025-05-25 20:04:34,166 - BERTopic - Topic reduction - Reduced number of topics from 17 to 13


In [67]:
topic_info = topic_model.get_topic_info()
print(topic_info[["Topic", "Count", "Name"]])

    Topic  Count                                Name
0      -1    764             -1_man_world_time_years
1       0    177              0_agent_man_world_case
2       1     99           1_woman_mother_home_house
3       2     64            2_planet_earth_crew_face
4       3     51            3_couple_york_road_night
5       4     47          4_quest_warrior_ring_power
6       5     45               5_story_film_rome_man
7       6     39               6_car_fight_drug_turn
8       7     31  7_relationship_school_decides_john
9       8     24       8_group_strangers_room_escape
10      9     22            9_princess_far_land_true
11     10     21     10_decision_begins_love_greater
12     11     20     11_attempts_success_years_night


In [68]:
df["Topic"] = topics
df["Topic_Prob"] = [p.max() if isinstance(p, (list, np.ndarray)) else None for p in probs]

for topic in sorted(df["Topic"].unique()):
    print(f"\nCluster {topic} — {len(df[df['Topic']==topic])} documenti")
    print(topic_model.get_topic_info()[topic_model.get_topic_info()["Topic"] == topic]['Name'].iloc[0])
    display(df[df["Topic"] == topic][["Title", "Plot", "Topic_Prob"]].sort_values(by="Topic_Prob", ascending=False).head(10))


Cluster -1 — 764 documenti
-1_man_world_time_years


,Title,Plot,Topic_Prob
650,The Suicide Squad,"Supervillains Harley Quinn, Bloodsport, Peacem...",0.352690
1045,The Cat o' Nine Tails,A newsman works with a blind puzzle-solver to ...,0.345286
976,Anatomy of a Fall,"A woman is suspected of her husband's murder, ...",0.328354
353,Licence to Kill,After capturing the notorious drug lord Franz ...,0.326357
1051,Hulk,"Bruce Banner, a genetics researcher with a tra...",0.326112
141,Despicable Me 2,Gru is recruited by the Anti-Villain League to...,0.325376
910,The Avengers,When an unexpected enemy emerges and threatens...,0.321899
1112,A History of Violence,An average family is thrust into the spotlight...,0.309480
1183,Suicide Squad,"From DC Comics comes the Suicide Squad, an ant...",0.307920
663,M,"In this classic German thriller, Hans Beckert,...",0.300960



Cluster 0 — 177 documenti
0_agent_man_world_case


,Title,Plot,Topic_Prob
40,Unknown,"A man awakens from a coma, only to discover th...",1.0
122,Deadpool 2,Wisecracking mercenary Deadpool battles the ev...,1.0
64,X2,Professor Charles Xavier and his team of genet...,1.0
70,Die Another Day,James Bond is sent to investigate the connecti...,1.0
113,Fight Club,A ticking-time-bomb insomniac and a slippery s...,1.0
102,Moonraker,After Drax Industries' Moonraker space shuttle...,1.0
106,Rear Window,A wheelchair-bound photographer spies on his n...,1.0
98,Strangers on a Train,"Having met on a train, a smooth-talking psycho...",1.0
331,The Judge,A successful lawyer returns to his hometown fo...,1.0
219,Rope,Two young men attempt to prove they committed ...,1.0



Cluster 1 — 99 documenti
1_woman_mother_home_house


...

,Title,Plot,Topic_Prob
175,Barbie,Barbie and Ken are having the time of their li...,1.0
201,The Magic Flute,The Queen of the Night enlists a handsome prin...,1.0
235,Frozen,Young princess Anna of Arendelle dreams about ...,1.0
238,Sleeping Beauty,Cursed to die by the evil fairy Maleficent whe...,1.0
379,Snow White and the Seven Dwarfs,"A beautiful girl, Snow White, takes refuge in ...",1.0
390,Beauty and the Beast,"Follow the adventures of Belle, a bright young...",1.0
394,Bambi,Bambi's tale unfolds from season to season as ...,1.0
430,Shrek,It ain't easy bein' green -- especially if you...,1.0
481,Shrek Forever After,A bored and domesticated Shrek pacts with deal...,1.0
484,Aladdin,"In the boorish city of Agrabah, kind-hearted s...",1.0



Cluster 10 — 21 documenti
10_decision_begins_love_greater


,Title,Plot,Topic_Prob
46,Humanist Vampire Seeking Consenting Suicidal P...,Sasha is a young vampire with a serious proble...,1.0
65,After We Fell,"Just as Tessa's life begins to become unglued,...",1.0
341,After We Collided,Tessa finds herself struggling with her compli...,1.0
351,The Twilight Saga: Breaking Dawn – Part 2,"After the birth of Renesmee, the Cullens gathe...",1.0
354,Black Swan,"The story of Nina, a ballerina in a New York C...",1.0
442,After Ever Happy,As a shocking truth about a couple's families ...,1.0
475,After Everything,Besieged by writer’s block and the crushing br...,1.0
498,Roman Holiday,"Overwhelmed by her suffocating schedule, touri...",1.0
536,The Outside,"Longing to fit in at work, awkward Stacey begi...",1.0
543,Repulsion,Beautiful young manicurist Carole suffers from...,1.0



Cluster 11 — 20 documenti
11_attempts_success_years_night


,Title,Plot,Topic_Prob
57,Death Sentence,Nick Hume is a mild-mannered executive with a ...,1.0
108,Soul,Joe Gardner is a middle school teacher with a ...,1.0
250,The King of Comedy,Aspiring comic Rupert Pupkin attempts to achie...,1.0
319,Network,When veteran anchorman Howard Beale is forced ...,1.0
374,The Social Network,"In 2003, Harvard undergrad and computer progra...",1.0
388,The Intern,70-year-old widower Ben Whittaker has discover...,1.0
439,Green Book,"Tony Lip, a bouncer in 1962, is hired to drive...",1.0
482,Shaun of the Dead,"Shaun lives a supremely uneventful life, which...",1.0
492,The Devil Wears Prada,Andy moves to New York to work in the fashion ...,1.0
615,17 Again,"On the brink of a midlife crisis, 30-something...",1.0
